In [ ]:
from dotenv import load_dotenv
import os
import sys
import pandas as pd
import logging

from core.data_structures.candles import Candles

logging.getLogger("asyncio").setLevel(logging.CRITICAL)

load_dotenv()
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [ ]:
from core.data_sources import CLOBDataSource

connector_name = "binance_perpetual"
interval = "1h"
days = 8

# Get trading rules and candles
clob = CLOBDataSource()

In [ ]:
from core.services.timescale_client import TimescaleClient

ts_client = TimescaleClient(host=os.getenv("TIMESCALE_HOST", "localhost"))
await ts_client.connect()

available_pairs: tuple = await ts_client.get_available_pairs()
trading_pairs = [trading_pair for connector_name, trading_pair in available_pairs]
trading_pairs

In [ ]:
all_candles = []
candles = await clob.get_candles_batch_last_days(connector_name, trading_pairs, interval, days, batch_size=len(trading_pairs))

In [ ]:
# Define intervals in hours
intervals = {"4h": 4, "8h": 8, "1d": 24, "3d": 72, "7d": 168}

# List to collect summarized data for each trading pair
summary_data = []

for candle in candles:
    df = candle.data.copy()  # Extract the DataFrame
    trading_pair = candle.trading_pair  # Get the trading pair name

    # Create a dictionary to store results for this trading pair
    trading_pair_summary = {"trading_pair": trading_pair}

    for interval, hours in intervals.items():
        # Subset the last `hours` worth of data
        subset_df = df.tail(hours)

        # Calculate metrics
        trading_pair_summary[f"{interval}_min"] = subset_df['low'].min()
        trading_pair_summary[f"{interval}_max"] = subset_df['high'].max()
        trading_pair_summary[f"{interval}_volume"] = subset_df['quote_asset_volume'].sum()

    # Append the summarized data
    summary_data.append(trading_pair_summary)

# Convert the summary data to a DataFrame
summary_df = pd.DataFrame(summary_data)
summary_df

In [ ]:
from plotly.subplots import make_subplots
import pandas as pd
import plotly.graph_objects as go

# Reshape the data to long format
data_long = pd.melt(
    summary_df,
    id_vars=["trading_pair"],  # Columns to keep
    value_vars=[col for col in summary_df.columns if "min" in col or "max" in col or "volume" in col],  # Columns to reshape
    var_name="interval_price",  # Name for reshaped columns
    value_name="price",  # Name for reshaped values
)

# Extract interval and price type from the column names
data_long["interval"] = data_long["interval_price"].str.extract(r"(\d+[hd])")  # Extract interval (e.g., "4h")
data_long["price_type"] = data_long["interval_price"].str.extract(r"(min|max)")  # Extract type (min or max)

# Calculate the spread (max - min) and the percentage spread
spreads = summary_df.copy()
for col in [col for col in spreads.columns if "max" in col]:
    interval = col.split("_")[0]
    spreads[f"{interval}_spread"] = spreads[f"{interval}_max"] - spreads[f"{interval}_min"]
    spreads[f"{interval}_spread_pct"] = (spreads[f"{interval}_spread"] / spreads[f"{interval}_min"]) * 100

spread_long = pd.melt(
    spreads,
    id_vars=["trading_pair"],
    value_vars=[col for col in spreads.columns if "spread" in col],
    var_name="interval_spread",
    value_name="spread",
)
spread_long["interval"] = spread_long["interval_spread"].str.extract(r"(\d+[hd])")
spread_long["metric"] = spread_long["interval_spread"].str.extract(r"(spread_pct|spread)")

# All trading pairs
all_pairs = summary_df["trading_pair"].unique()
rows = (len(all_pairs) + 3) // 4  # Calculate rows needed for 4 per row

# Create the faceted plot
fig = make_subplots(
    rows=rows, cols=4, shared_xaxes=True,
    subplot_titles=all_pairs,  # Titles for each subplot
    specs=[[{"secondary_y": True}] * 4 for _ in range(rows)]  # Add secondary y-axis for each subplot
)

for idx, trading_pair in enumerate(all_pairs, 1):
    row, col = divmod(idx - 1, 4)  # Calculate row and column
    row += 1  # Plotly rows are 1-indexed
    col += 1  # Plotly cols are 1-indexed

    # Filter data for the current trading pair
    data = data_long.loc[data_long["trading_pair"] == trading_pair]
    spread_data = spread_long.loc[
        (spread_long["trading_pair"] == trading_pair) & (spread_long["metric"] == "spread")
    ]
    spread_pct_data = spread_long.loc[
        (spread_long["trading_pair"] == trading_pair) & (spread_long["metric"] == "spread_pct")
    ]

    # Add the filled area between min and max
    fig.add_trace(
        go.Scatter(
            x=data.loc[data["price_type"] == "max", "interval"],
            y=data.loc[data["price_type"] == "max", "price"],
            name="Max",
            line=dict(color="white", width=0),  # Invisible line for fill
            fillcolor="rgba(0, 0, 255, 0.2)",  # Blue fill with opacity
            showlegend=False,  # Hide legend for individual subplots
        ),
        row=row, col=col, secondary_y=False
    )

    # Add min series
    fig.add_trace(
        go.Scatter(
            x=data.loc[data["price_type"] == "min", "interval"],
            y=data.loc[data["price_type"] == "min", "price"],
            name="Min",
            line=dict(color="white"),
            fill="tonexty",
            showlegend=False  # Hide legend for individual subplots
        ),
        row=row, col=col, secondary_y=False
    )

    # Add max series
    fig.add_trace(
        go.Scatter(
            x=data.loc[data["price_type"] == "max", "interval"],
            y=data.loc[data["price_type"] == "max", "price"],
            name="Max",
            line=dict(color="white"),
            showlegend=False  # Hide legend for individual subplots
        ),
        row=row, col=col, secondary_y=False
    )

    # Add spread series with percentage labels
    fig.add_trace(
        go.Scatter(
            x=spread_data["interval"],
            y=spread_data["spread"],
            name="Spread",
            line=dict(color="yellow", dash="dot", width=2),  # Strong, contrasting color
            text=[f"{pct:.2f} %" for pct in spread_pct_data["spread"]],
            textfont=dict(color="yellow"),
            textposition="top center",
            mode="lines+text",  # Add text labels along the line
            showlegend=False  # Hide legend for individual subplots
        ),
        row=row, col=col, secondary_y=True
    )

for i in range(1, len(fig.layout.annotations) + 1):  # Loop through all subplot indices
    fig.update_xaxes(
        row=(i + 3) // 4,  # Calculate row
        col=(i - 1) % 4 + 1,  # Calculate column
        title="Interval",
        categoryorder="array",
        categoryarray=["7d", "3d", "1d", "8h", "4h"],  # Desired order
    )

# Update layout for better readability
fig.update_layout(
    height=300 * rows,  # Adjust height based on the number of rows
    yaxis_title="Price",
    yaxis2_title="Spread",
    legend_title="Metrics",
    margin=dict(t=50, l=50, r=50, b=50),
    font=dict(size=12),
)

# Display the plot
fig.show()

In [ ]:
base_config = {
    "controller_name": "grid_strike_grid_component",
    "controller_type": "generic",
    "manual_kill_switch": None,
    "candles_config": [],
    "position_mode": "HEDGE",
    "connector_name": connector_name,
    "max_open_orders": 3,
    "max_orders_per_batch": 1,
    "order_frequency": 7,
    "activation_bounds": 0.002,
    "total_amount_quote": 2000,
    "min_spread_between_orders": 0.0002,
    "min_order_amount_quote": 11,
    "time_limit": 60 * 60 * 24 * 3,
    "leverage": 50,
    "triple_barrier_config": {
        "open_order_type": 3,
        "stop_loss": None,
        "stop_loss_order_type": 1,
        "take_profit": 0.0008,
        "take_profit_order_type": 3,
        "time_limit": 21600,
        "time_limit_order_type": 1,
        "trailing_stop": {
            "activation_price": 0.03,
            "trailing_delta": 0.005
        },
    }
}

In [ ]:
trading_pairs = {"SUI-USDT": 2, "WLD-USDT": 2}
limit_price_risk = 0.015
candidates = summary_df[["trading_pair", "7d_min", "7d_max"]]
candidates.columns = ["trading_pair", "start_price", "end_price"]
candidates = candidates.to_dict(orient="records")
selections = [candidate for candidate in candidates if candidate["trading_pair"] in trading_pairs.keys()]
for selection in selections:
    selection.update({"side": trading_pairs[selection["trading_pair"]]})
    selection["limit_price"] = selection["start_price"] * (1 - limit_price_risk) if selection["side"] == 1 else selection["end_price"] * (1 + limit_price_risk)
selections

In [ ]:
for selection in selections:
    candle: Candles = [candle for candle in candles if candle.trading_pair == selection["trading_pair"]][0]
    fig = candle.fig()
    fig.add_hline(selection["start_price"])
    fig.add_hline(selection["limit_price"])
    fig.add_hline(selection["end_price"])
    fig.show()

In [ ]:
name = "gridrupman"
version = "0.2"
deploy_configs = []
for selection in selections:
    base = base_config.copy()
    selection["id"] = f"{name}-{connector_name.replace('_', '')}-{selection['trading_pair'].lower().split('-')[0]}-{'long' if selection['side'] == 1 else 'short'}_{version}"
    base.update(selection)
    deploy_configs.append(base)
deploy_configs

In [ ]:
from core.services.backend_api_client import BackendAPIClient

backend_api = BackendAPIClient(host=os.getenv("TRADING_HOST"), username="admin", password="admin")
for i, config in enumerate(deploy_configs, start=1):
    print(f"Uploading config {i} of {len(deploy_configs)}")
    await backend_api.add_controller_config(config)

In [ ]:
config_ids = [config["id"] + ".yml" for config in deploy_configs]
config_ids

In [ ]:
await backend_api.deploy_script_with_controllers(bot_name="okxgridrupman",
                                                 controller_configs=config_ids,
                                                 image_name="hummingbot/hummingbot:grid_executor",
                                                 credentials="okx-sere")